# Proyecto Individual 



Este programa está enfocado en la descarga de data referente a las plataformas de Disney Plus, Netflix, Amazon y Hulu. Realiza un proceso de ETL para poder pasar los datos a SQL y tener una mejor visualización. 

In [14]:
"""Se descargan librerias necesarias para el trabajo a realizar"""
import pandas as pd
#from sqlalchemy import create_engine
#from sqlalchemy.engine import URL
#import unidecode

In [15]:
# Se realiza ingesta de los datos 

amazonUrl = 'https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/amazon_prime_titles.csv'
netflixUrl = 'https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/netflix_titles.json'
disneyUrl = 'https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/disney_plus_titles.csv'
huluUrl = 'https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/hulu_titles.csv'


def ingestadorDeData (url = str ,tipo = str):
    """Esta función genera un DataFrame a partir de un URL con la data necesaria para poder descargar."""

    if (tipo == 'csv'):
        data = pd.read_csv(url, )
    elif (tipo == 'json'):
        data = pd.read_json(url, )
    else :
        print('el tipo requerido es csv o json, volver a intentar')
        data = None
    return data; 


# se llama a la función ingestadorDeData para cada uno de los URL 

amazonData = ingestadorDeData(amazonUrl,'csv')
netflixData = ingestadorDeData(netflixUrl,'json')
disneyData = ingestadorDeData(disneyUrl,'csv')
huluData = ingestadorDeData(huluUrl,'csv')

In [16]:
# Aqui decidí que se debía agregar una columna para poder tener la información referente a que plataforma estamos haciendo referencia en esa fila

amazonData['platform'] = 'Amazon'
netflixData['platform'] = 'Netflix'
huluData['platform'] = 'Hulu'
disneyData['platform'] = 'Disney'

In [17]:
amazonData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
 12  platform      9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB


A partir de este decidí realizar una concatenación de los datos  para poder empezar a analizarlos y proseguir con algo de ETL

In [18]:
totaldata = pd.concat([amazonData,netflixData,huluData,disneyData],levels= ['platform'])

In [19]:
# una vez todos los datos están en el mismo DataFrame agrego un id único para cada una de las peliculas registradas 

totaldata['id']= range(1,len(totaldata)+1)

In [20]:
 
def deleteadorDeColumnas(data = pd.DataFrame):
    """Esta función recibe como parámetro un DataFrame y elimina las columnas:
    'director','country','date_added','rating','description'
     """

    if (type(data)== pd.DataFrame):
        cadenaTitulos = ('director','country','date_added','rating','description','show_id')
        data1 = data
        # Este for se encarga de dropear todas las columnas que haya dentro de la lista cadenaTitulos
        for numero,columna in enumerate(cadenaTitulos):
            if columna in data1.columns:
                data1.drop(labels = columna, axis  =1, inplace = True)
        return data1
    else: 
        print('El tipo de dato debe ser DataFrame')

# se manda llamar a la función deleteadorDeColumnas para cada uno de los DataFrames
totaldata = deleteadorDeColumnas(totaldata)


In [21]:
# Se genera función para cambiar los nombres de las columnas 

def cambiadorDeColumnas(data = pd.DataFrame):
    """Esta función recibe como parámetro un DataFrame para cambiarle los nombres a las columnas release_year y listed_in"""

    if (type(data)!= pd.DataFrame):
        print('Debe cargar un DataFrame')
    else:
        data.rename(columns = {'release_year': 'releaseYear',
              'listed_in': 'listedIn'}, inplace = True)
    return data

totaldata = cambiadorDeColumnas(totaldata)


In [22]:
totaldata

,type,title,cast,releaseYear,duration,listedIn,platform,id
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113 min,"Comedy, Drama",Amazon,1
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110 min,"Drama, International",Amazon,2
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74 min,"Action, Drama, Suspense",Amazon,3
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69 min,Documentary,Amazon,4
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45 min,"Drama, Fantasy",Amazon,5
...,...,...,...,...,...,...,...,...
1445,Movie,X-Men Origins: Wolverine,"Hugh Jackman, Liev Schreiber, Danny Huston, wi...",2009,108 min,"Action-Adventure, Family, Science Fiction",Disney,22994
1446,Movie,Night at the Museum: Battle of the Smithsonian,"Ben Stiller, Amy Adams, Owen Wilson, Hank Azar...",2009,106 min,"Action-Adventure, Comedy, Family",Disney,22995
1447,Movie,Eddie the Eagle,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...",2016,107 min,"Biographical, Comedy, Drama",Disney,22996
1448,Movie,Bend It Like Beckham,"Parminder Nagra, Keira Knightley, Jonathan Rhy...",2003,112 min,"Buddy, Comedy, Coming of Age",Disney,22997


In [23]:
# aqui genero la función necesaria para poder separar los actores de la columna cast 

splitedcast = totaldata.cast.str.split(',', expand = True)
splitedcast['idActor'] = range(1,len(splitedcast)+1)
splitedcast

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,idActor
0,Brendan Gleeson,Taylor Kitsch,Gordon Pinsent,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,1
1,Mahesh Manjrekar,Abhay Mahajan,Sachin Khedekar,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,2
2,Tom Sizemore,Lorenzo Lamas,Robert LaSardo,Richard Jones,Yancey Arias,Noel Gugliemi,None,None,None,None,...,None,None,None,None,None,None,None,None,None,3
3,Interviews with: Pink,Adele,Beyoncé,Britney Spears,Christina Aguilera,more!,None,None,None,None,...,None,None,None,None,None,None,None,None,None,4
4,Harry Dean Stanton,Kieran O'Brien,George Costigan,Amanda Dickinson,Alison Steadman,Grant Bardsley,Bill Moody,Matthew Scurfield,None,None,...,None,None,None,None,None,None,None,None,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,Hugh Jackman,Liev Schreiber,Danny Huston,will.i.am,Lynn Collins,Kevin Durand,None,None,None,None,...,None,None,None,None,None,None,None,None,None,22994
1446,Ben Stiller,Amy Adams,Owen Wilson,Hank Azaria,Christopher Guest,Alain Chabat,None,None,None,None,...,None,None,None,None,None,None,None,None,None,22995
1447,Tom Costello,Jo Hartley,Keith Allen,Dickon Tolson,Jack Costello,Taron Egerton,None,None,None,None,...,None,None,None,None,None,None,None,None,None,22996
1448,Parminder Nagra,Keira Knightley,Jonathan Rhys-Meyers,Anupam Kher,Archie Panjabi,Shaznay Lewis,None,None,None,None,...,None,None,None,None,None,None,None,None,None,22997


In [24]:
# aqui genero la función necesaria para poder separar las categorias de la columna ListedIn 

splitedListedIn= totaldata.listedIn.str.split(',', expand = True)
splitedListedIn['idCategoria'] = range(1,len(splitedListedIn)+1)
splitedListedIn.rename(columns = {
    0: 'Categoria1',
    1: 'Categoria2',
    2: 'Categoria3',
    3: 'Categoria4',
    4: 'Categoria5'
})



,Categoria1,Categoria2,Categoria3,Categoria4,Categoria5,idCategoria
0,Comedy,Drama,None,None,None,1
1,Drama,International,None,None,None,2
2,Action,Drama,Suspense,None,None,3
3,Documentary,None,None,None,None,4
4,Drama,Fantasy,None,None,None,5
...,...,...,...,...,...,...
1445,Action-Adventure,Family,Science Fiction,None,None,22994
1446,Action-Adventure,Comedy,Family,None,None,22995
1447,Biographical,Comedy,Drama,None,None,22996
1448,Buddy,Comedy,Coming of Age,None,None,22997


In [25]:
# En esta sección se crea un motor para poder compartir la base de datos a SQL 
#url_object = URL.create(
#    drivername = "mysql+pymysql",
#    username="root",
#    password="Leonardo1",  
#    host="localhost",
#    port = "3306",
#    database="PI01Henry"
#)

#engine = create_engine(url_object, encoding = 'latin-1')

In [26]:
# Decido mandar 3 tablas a SQL para empezar con las consultas necesarias

#totaldata.to_sql('TotalData',con=engine,index=False, if_exists='replace')
#splitedcast.to_sql('listadoActores',con=engine,index=True, if_exists='replace')
#splitedListedIn.to_sql('listadoCategorias',con=engine,index=True, if_exists='replace')

In [28]:
# Mandamos las 3 tablas a 1 csv para poder subirlas a las api

totaldata.to_csv('Data')
splitedcast.to_csv('Actores')
splitedListedIn.to_csv('Categorias')

In [11]:
pip freeze

anyio==3.6.2
asttokens==2.0.7
backcall==0.2.0
certifi==2022.9.24
cffi==1.15.1
charset-normalizer==2.1.1
click==8.1.3
colorama==0.4.5
contourpy==1.0.5
cryptography==38.0.1
cycler==0.11.0
debugpy==1.6.2
decorator==5.1.1
entrypoints==0.4
et-xmlfile==1.1.0
executing==0.9.1
fastapi==0.88.0
fonttools==4.37.4
greenlet==1.1.3.post0
h11==0.14.0
idna==3.4
ipykernel==6.15.1
ipython==8.4.0
jedi==0.18.1
joblib==1.2.0
jupyter-client==7.3.4
jupyter-core==4.11.1
kiwisolver==1.4.4
matplotlib==3.6.0
matplotlib-inline==0.1.3
mysql==0.0.3
mysqlclient==2.1.1
nest-asyncio==1.5.5
numpy==1.23.2
openpyxl==3.0.10
packaging==21.3
pandas==1.4.4
parso==0.8.3
pickleshare==0.7.5
Pillow==9.2.0
prompt-toolkit==3.0.30
psutil==5.9.1
pure-eval==0.2.2
pycparser==2.21
pydantic==1.10.2
Pygments==2.12.0
PyMySQL==1.0.2
pyparsing==3.0.9
python-dateutil==2.8.2
pytz==2022.2.1
pywin32==304
pyzmq==23.2.0
requests==2.28.1
scikit-learn==1.1.3
scipy==1.9.1
seaborn==0.12.1
six==1.16.0
sklearn==0.0.post1
sniffio==1.3.0
SQLAlchemy==1.4.